## Aula 03 - Filtragem Baseada em Conteúdo - Exercícios

In [1]:
import pandas as pd
import numpy as np
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN
from sklearn.neural_network import MLPClassifier, MLPRegressor


### Importar base de dados

In [2]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar (7).gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv


In [3]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [4]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [5]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
map_title = {}

for _, row in df.iterrows():
    map_title[row.movieId] = row.title


In [6]:
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,18,34,Gregory Peck,1329962459
1,18,34,need to own,1329962471
2,18,34,romantic comedy,1329962476
3,18,34,Rome,1329962490
4,18,34,royalty,1329962474


### Divisão da base em treino e teste

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)

***Exercício 01:*** Aplique a filtragem baseada em conteúdo (ItemAttributeKNN do CaseRecommender) com as tags associadas aos filmes da base. Utilize Jaccard como métrica de similaridade e k=5 vizinhos para predição.

Documentação do ItemAttributeKNN: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [8]:
# Saving the data to csv files
train.to_csv('train.csv', index=False, sep='\t', header=False)
test.to_csv('test.csv', index=False, sep='\t', header=False)
movies_tags[['movieId', 'tag']].to_csv('tags.csv', index=False, sep='\t', header=False)

In [9]:
movies_tags

,userId,movieId,tag,timestamp_y
0,18,34,Gregory Peck,1329962459
1,18,34,need to own,1329962471
2,18,34,romantic comedy,1329962476
3,18,34,Rome,1329962490
4,18,34,royalty,1329962474
...,...,...,...,...
6269,11075,129,ambitious,1423178287
6270,11075,19,nonlinear,1407271610
6271,11075,19,Quentin Tarantino,1407271608
6272,11075,129,Self-Indulgent,1423178219


In [10]:
# Obter a lista de gêneros de um item
def get_tags(itemId):
    if itemId not in movies_tags['movieId'].values:
        return []
    return movies_tags.loc[(movies_tags['movieId']==itemId),'tag'].tolist()

get_tags(0)

['adapted from:book',
 'war',
 'marooned',
 'stranded',
 'alien',
 'aliens',
 'enemy',
 'aliens',
 'spaceflight',
 'space',
 'sci-fi',
 'stranded',
 'war',
 'aliens',
 'heavy handed',
 'cheesy',
 'massacred the book',
 'aliens',
 'Dennis Quaid',
 'space',
 'Louis Gossett Jr.',
 'science fiction',
 'future',
 'sci-fi',
 'aliens']

In [11]:
from math import pow, sqrt

def similarity_score(itemId1, itemId2):
    '''
    itemId1 & itemId2 : ids dos dois itens cuja similaridade será computada
    '''
    # Obter os gêneros de cada item.
    genre_list1 = get_tags(itemId1)
    genre_list2 = get_tags(itemId2)
    common_genres = list(set(genre_list1) & set(genre_list2))
    if len(common_genres) == 0:
        return 0
    
    # Calcular Jaccard
    return len(common_genres) / len(set(genre_list1 + genre_list2))

similarity_score(4, 5)

0.036585365853658534

In [12]:
df.movieId.nunique()

417

In [13]:
# get similarity matrix for all items
def get_similarity_matrix():
    n_items = df.movieId.max() + 1
    similarity_matrix = np.zeros((n_items, n_items))
    for i in range(n_items):
        for j in range(n_items):
            similarity_matrix[i][j] = similarity_score(i, j)
        
    return similarity_matrix

In [14]:
def np_to_pd_matrix(matrix):
    # Find the non-zero indices
    non_zero_indices = np.nonzero(matrix)

    # Extract the row indices, column indices, and values
    rows = non_zero_indices[0]
    cols = non_zero_indices[1]
    values = matrix[non_zero_indices]

    # Create a pandas DataFrame with tuples (row_id, column_id, value)
    return pd.DataFrame({
        'row_id': rows,
        'column_id': cols,
        'value': values
    })

In [15]:
# Calculate the similarity matrix
sim_matrix = get_similarity_matrix()

sim_matrix

array([[1.        , 0.04347826, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04347826, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [16]:
sparse_sim_matrix = np_to_pd_matrix(sim_matrix)

sparse_sim_matrix

,row_id,column_id,value
0,0,0,1.000000
1,0,1,0.043478
2,0,4,0.020833
3,0,5,0.029412
4,0,10,0.022472
...,...,...,...
3426,375,262,0.142857
3427,375,372,0.083333
3428,375,375,1.000000
3429,382,382,1.000000


In [17]:
# Save the sparse similarity matrix to a CSV file
sparse_sim_matrix.to_csv('sim_matrix.csv', index=False, sep='\t', header=False)

In [18]:
ItemAttributeKNN(
    'train.csv',
    'test.csv', 
    metadata_file='tags.csv', 
    as_similar_first=True,
    k_neighbors=5,
    output_file='predictions.csv',
    similarity_file='sim_matrix.csv'
).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.453679 sec
>> metadata:: 231 items and 1979 metadata (6274 interactions) | sparsity:: 98.63%
prediction_time:: 0.283286 sec
Eval:: MAE: 0.729791 RMSE: 0.959396 


### Preparação para o exercício 2 - Download e extração de metadados multimídia

In [19]:
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-features.tar.gz
! tar -xvzf ml-20m-features.tar.gz

100% [......................................................] 5996435 / 5996435
Saved under ml-20m-features.tar (5).gz
features/
features/._m4infus_max_histogram_300_sn.arq
features/m4infus_max_histogram_300_sn.arq
features/._mm_avg_histogram_100_sn.arq
features/mm_avg_histogram_100_sn.arq
features/._visual_histogram_100_sn.arq
features/visual_histogram_100_sn.arq
features/._visual_histogram_50_sn.arq
features/visual_histogram_50_sn.arq
features/._aural_histogram_50.arq
features/aural_histogram_50.arq
features/._mm_max_histogram_300.arq
features/mm_max_histogram_300.arq
features/._m4infus_max_histogram_50.arq
features/m4infus_max_histogram_50.arq
features/._mm_max_histogram_100.arq
features/mm_max_histogram_100.arq
features/._mm_max_histogram_50_sn.arq
features/mm_max_histogram_50_sn.arq
features/._visual_histogram_100.arq
features/visual_histogram_100.arq
features/._visual_histogram_300.arq
features/visual_histogram_300.arq
features/._aural_histogram_100_sn.arq
features/aural_histogra

In [20]:
import pickle

with open('./features/visual_histogram_50.arq', 'rb') as arq_visualHistograms:
    visualHistograms = pickle.load(arq_visualHistograms)
print('No. movies: ' + str(len(visualHistograms)))
print('Size of each word: ' + str(len(visualHistograms[0])))

No. movies: 433
Size of each word: 50


In [21]:
visualHistograms[0]

array([0.00948992, 0.00237248, 0.00355872, 0.01304864, 0.05338078,
       0.00711744, 0.05456702, 0.00355872, 0.0059312 , 0.00355872,
       0.0059312 , 0.00474496, 0.00711744, 0.04507711, 0.07591934,
       0.00355872, 0.1316726 , 0.05338078, 0.00830368, 0.        ,
       0.01304864, 0.00474496, 0.02253855, 0.00355872, 0.02728351,
       0.00237248, 0.02016607, 0.00830368, 0.0059312 , 0.00237248,
       0.00355872, 0.04033215, 0.01067616, 0.00237248, 0.00830368,
       0.05931198, 0.0059312 , 0.01067616, 0.02965599, 0.01423488,
       0.00711744, 0.02609727, 0.        , 0.00237248, 0.01423488,
       0.03084223, 0.02016607, 0.00118624, 0.08778173, 0.02253855])

***Exercício 02:*** Como visto, o algoritmo ItemAttributeKNN pode ser usado com diferentes tipos de metadados, como gêneros, tags e palavras no geral. Mais do que isso, podemos adaptá-lo também para que a similaridade entre itens seja feita com base em informações multimídia, como imagens, áudio, etc. 

A base de dados utilizada até o momento, ml-20m-compact.tar.gz possui, além das interações de usuários com filmes, uma série de arquivos que contém informações multimídia que foram extraídas dos trailers de cada filme. Esses arquivos estão condensados no zip ml-20m-features.tar.gz, o qual foi feito o download e extraído acima. 

Considere por exemplo o arquivo visual_histogram_50.arq. Ele possui 433 vetores (no. de filmes) de tamanho 50. Podemos pensar que cada vetor desse representa informações visuais (cor, brilho, imagem, etc.) que foram extraídas dos trailers de cada filme. 

Sua tarefa é usar esses vetores de características visuais no cálculo de similaridade entre os filmes, e em seguida, aplicar essas similaridades no algoritmo ItemAttributeKNN para gerar recomendações. 

Dica 1: para calcular a similaridade entre dois vetores pode-se usar o ângulo de cosseno (vide https://en.wikipedia.org/wiki/Cosine_similarity). 

Dica 2: é possível passar para o algoritmo ItemAttributeKNN a matriz de similaridade entre itens, por meio do parâmetro similarity_file=arquivo. Veja em: https://github.com/caserec/CaseRecommender/blob/master/caserec/recommenders/rating_prediction/item_attribute_knn.py

In [22]:
# Convert to numpy array
data = np.array(visualHistograms)

# Normalize each row (item) to unit vector (magnitude 1)
norms = np.linalg.norm(data, axis=1, keepdims=True)
normalized_data = data / norms

# Calculate cosine similarity matrix (dot product of normalized vectors)
cosine_similarity_matrix = np.dot(normalized_data, normalized_data.T)

cosine_similarity_matrix

/tmp/ipykernel_537647/3431986793.py:6: RuntimeWarning: invalid value encountered in divide
  normalized_data = data / norms


array([[1.        , 0.71195759, 0.48778751, ..., 0.51796631, 0.62916574,
        0.32275161],
       [0.71195759, 1.        , 0.56565304, ..., 0.64526469, 0.63512788,
        0.24971473],
       [0.48778751, 0.56565304, 1.        , ..., 0.6055329 , 0.52081767,
        0.67566875],
       ...,
       [0.51796631, 0.64526469, 0.6055329 , ..., 1.        , 0.68601105,
        0.28055194],
       [0.62916574, 0.63512788, 0.52081767, ..., 0.68601105, 1.        ,
        0.29672386],
       [0.32275161, 0.24971473, 0.67566875, ..., 0.28055194, 0.29672386,
        1.        ]])

In [23]:
pd_cosine_similarity_matrix = np_to_pd_matrix(cosine_similarity_matrix)
pd_cosine_similarity_matrix

,row_id,column_id,value
0,0,0,1.000000
1,0,1,0.711958
2,0,2,0.487788
3,0,3,0.247959
4,0,4,NaN
...,...,...,...
187222,432,428,0.582932
187223,432,429,0.224129
187224,432,430,0.280552
187225,432,431,0.296724


In [24]:
pd_cosine_similarity_matrix.to_csv('cosine_sim_matrix.csv', index=False, sep='\t', header=False)

In [25]:
ItemAttributeKNN(
    'train.csv',
    'test.csv', 
    metadata_file='tags.csv', 
    as_similar_first=True,
    k_neighbors=5,
    output_file='predictions.csv',
    similarity_file='cosine_sim_matrix.csv'
).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.426803 sec
>> metadata:: 231 items and 1979 metadata (6274 interactions) | sparsity:: 98.63%
prediction_time:: 0.279818 sec
Eval:: MAE: 0.729791 RMSE: 0.959396 


***Exercício 03:*** Implementar uma função que retorna a probabilidade de um item ser relevante para um usuário, considerando os gêneros dos filmes. Utilize métodos probabilísticos. 
- A partir do conjunto de treinamento, obter todas as interações do usuário u.
- Rotular as notas desse usuário como: item relevante se nota >=3 e não relevante se nota < 3. 
- Dado um item do conjunto de teste, aplicar o Teorema de Bayes com suavização de Laplace. Utilizar os gêneros associados. 
- Retornar se o item é ou não relevante, e em seguida, comparar o resultado com a nota real que esse usuário deu para o item (disponível no conjunto de teste).

In [26]:
features = np.array(visualHistograms)
n_items = features.shape[0]
n_users = df.userId.max() + 1
n_features = len(features[0])
relevancy_threshold = 3.0

In [27]:
def is_relevant(itemId, userId):
    temp = df[(df['movieId'] == itemId) & (df['userId'] == userId)]
    return not temp.empty and temp['rating'].values[0] >= relevancy_threshold

is_relevant(0, 0), is_relevant(0, 1)

(True, False)

In [28]:
def get_labels(userId):
    return np.array([is_relevant(i + 1, userId) for i in range(n_items)])

In [29]:
def calculate_likelihoods(features, labels, label_value, smoothing=1):
    likelihoods = []
    for feature_index in range(n_features):
        # Count occurrences where the feature is 1 given the label
        count_feature_given_label = np.sum((features[:, feature_index] == 1) & (labels == label_value))
        count_label = np.sum(labels == label_value)
        
        # Apply Laplace smoothing
        likelihood = (count_feature_given_label + smoothing) / (count_label + 2 * smoothing)
        likelihoods.append(likelihood)
    
    return np.array(likelihoods)

In [30]:
def calculate_posterior_probabilities(item_features, likelihoods, prior):
    P_X_given_Y = np.prod([
        likelihood 
        if feature == 1 else (1 - likelihood) 
        for feature, likelihood in zip(item_features, likelihoods)
    ])
    
    P_Y_given_X = P_X_given_Y * prior
    
    return P_Y_given_X

In [31]:
def predict_label(item_index, features, likelihoods_R, likelihoods_N, P_R, P_N):
    item_features = features[item_index]
    
    P_R_given_X = calculate_posterior_probabilities(item_features, likelihoods_R, P_R)
    P_N_given_X = calculate_posterior_probabilities(item_features, likelihoods_N, P_N)
    
    return P_R_given_X, P_N_given_X

In [32]:
def prob_predict(userId, itemId):
    labels = get_labels(userId)
    likelihoods_R = calculate_likelihoods(features, labels, True)
    likelihoods_N = calculate_likelihoods(features, labels, False)
    P_R = np.sum(labels) / len(labels)
    P_N = 1 - P_R
    
    # Predict the label for the item
    result = predict_label(itemId, features, likelihoods_R, likelihoods_N, P_R, P_N)
    
    # Return the likelihood of the item being relevant
    return result[0]

In [33]:
prob_predict(1, 2)

0.0012829329497123971

In [34]:
# NOTE: Testing on a sample due to the high computational cost
prob_test_sample = test.sample(
    frac=0.01,
    random_state=42
)

# test prob_predict in the test set
def test_prob_predict():
    correct = 0
    for _, row in prob_test_sample.iterrows():
        userId = row['userId']
        itemId = row['movieId']
        prob = prob_predict(userId, itemId)
        if (prob >= 0.5 and row['rating'] >= relevancy_threshold) or (prob < 0.5 and row['rating'] < relevancy_threshold):
            correct += 1
    return correct / len(prob_test_sample)

In [35]:
test_prob_predict()

0.15748031496062992

***Exercício 04:*** No notebook de exemplos, existe uma implementação de Filtragem Baseada em Conteúdo usando Multi-Layer Perceptron como um regressor (MLPRegressor) que prevê a nota de cada usuário para filmes ainda não vistos. O algoritmo retorna uma lista de top K filmes com maiores notas. O treinamento é realizado utilizando as notas que o usuário deu para os filmes e seus respectivos gêneros.
- Usando a classe MLPClassifier (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html), implemente uma versão que classifica os filmes não vistos como relevante ou não-relevante.
- No conjunto de dados, realize a binarização das notas, de modo que notas acima de 3 são relevantes e notas abaixo de 3 são não-relevantes.
- Retorne os top k filmes mais relevantes.
- Para um usuário qualquer da base (ou algum outro usuário fictício), analise subjetivamente a qualidade das recomendações, comparando os modelos MLPRegressor e MLPClassifier. 

In [36]:
relevancy_threshold = 3.0

In [37]:
bin_df = df[['userId', 'movieId', 'rating']]
bin_df['bin_rating'] = bin_df['rating'] >= relevancy_threshold
bin_df

,userId,movieId,rating,bin_rating
0,0,0,5.0,True
1,0,1,4.5,True
2,0,2,4.0,True
3,0,3,2.0,False
4,0,4,5.0,True
...,...,...,...,...
190616,11089,22,5.0,True
190617,11089,10,5.0,True
190618,11089,2,4.0,True
190619,11089,21,4.5,True


In [38]:
# Create the pivot table with movieId as the index, ensuring that each tag presence is represented as 1
bin_tags = movies_tags.pivot_table(index='movieId', columns='tag', aggfunc=lambda x: 1, fill_value=0)

# Flatten the column index if it has multiple levels and ensure movieId is preserved
bin_tags.columns = bin_tags.columns.get_level_values(1)

# Reset the index to turn movieId back into a column
bin_tags = bin_tags.reset_index()


bin_tags.head()


tag,movieId,'It Should Have Been Kept A Secret',01.04.06,1.5,12-1-2007,14.01.06,16th century,1940s,1950s,1970s,...,woman in armour,words cannot explain how horrid this is,working class,working women,wormhole,writers,wry,younger Gandalf,yuppies,zest for life
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
def get_tags(movieId, binarized_df):
    """
    Returns a set of tags for the given movieId from the binarized DataFrame.
    
    :param movieId: The ID of the movie for which to retrieve tags.
    :param binarized_df: The binarized DataFrame where tags are columns.
    :return: A set of tags associated with the given movieId.
    """
    # Find the row corresponding to the movieId
    row = binarized_df[binarized_df['movieId'] == movieId]
    
    # If movieId is not found, return an empty set
    if row.empty:
        return set()
    
    # Get the tags where the value is 1
    tags = set(row.columns[row.iloc[0] == 1])
    
    return tags


# get_tags(0, bin_tags), get_tags(0, bin_tags) == set(movies_tags[movies_tags.movieId == 0].tag.values)

In [40]:
# Inicializar objetos para armazenar os modelos
user_models = {}

mlp_r_train_sample = train.sample(frac=0.001, random_state=42)

# Para cada usuário, treinar um modelo
for userId in mlp_r_train_sample['userId'].unique():
    # Selecionar filmes avaliados
    user_ratings = bin_df[bin_df['userId'] == userId]
    
    user_movies = user_ratings.merge(bin_tags, on='movieId')

    if len(user_movies) > 1:  # Treinar apenas se houver mais de um filme avaliado
        # Preparar os dados para treinamento da MLP
        X = user_movies.drop(['userId', 'movieId', 'rating', 'bin_rating'], axis=1)
        y = user_movies['rating']

        # Treinar a MLP
        mlp = MLPRegressor(hidden_layer_sizes=(50, 30), activation='relu', max_iter=1000, random_state=42)
        mlp.fit(X, y)

        # Salvar o modelo e o scaler do usuário
        user_models[userId] = mlp

In [41]:
# Inicializar objetos para armazenar os modelos
user_models_c = {}

mlp_r_train_sample = train.sample(frac=0.001, random_state=42)

# Para cada usuário, treinar um modelo
for userId in mlp_r_train_sample['userId'].unique():
    # Selecionar filmes avaliados
    user_ratings = bin_df[bin_df['userId'] == userId]
    
    user_movies = user_ratings.merge(bin_tags, on='movieId')

    if len(user_movies) > 1:  # Treinar apenas se houver mais de um filme avaliado
        # Preparar os dados para treinamento da MLP
        X = user_movies.drop(['userId', 'movieId', 'rating', 'bin_rating'], axis=1)
        y = user_movies['bin_rating']

        # Treinar a MLP
        mlp = MLPClassifier(hidden_layer_sizes=(50, 30), activation='relu', max_iter=1000, random_state=42)
        mlp.fit(X, y)

        # Salvar o modelo e o scaler do usuário
        user_models_c[userId] = mlp

In [42]:
def mlp_predict_one(userId, movieId):
    # Obter o modelo do usuário
    mlp_r = user_models.get(userId, None)
    mlp_c = user_models_c.get(userId, None)
    
    r_prediciton = None
    c_prediction = None
    
    X = bin_tags[bin_tags['movieId'] == movieId].drop(['movieId'], axis=1)
        
    if mlp_r is not None:        
        # Fazer a predição
        r_prediciton = mlp_r.predict(X)[0]
        
    if mlp_c is not None:
        # Fazer a predição
        c_prediction = mlp_c.predict(X)[0]

    return r_prediciton, c_prediction

def mlp_predict(userId):
    # Obter o modelo do usuário
    mlp_r = user_models.get(userId, None)
    mlp_c = user_models_c.get(userId, None)
    
    r_prediciton = None
    c_prediction = None
    
    X = bin_tags.drop(['movieId'], axis=1)

    if mlp_r is not None:
        r_prediciton = mlp_r.predict(X)
        
    if mlp_c is not None:
        c_prediction = mlp_c.predict(X)

    return r_prediciton, c_prediction

In [43]:
mlp_predict_one(0, 108)

(None, None)

In [44]:
mlp_predict_one(5928, 108)

(1.4859502078820206, False)

In [45]:
def recommend_movies(userId, k=10):
    r_prediciton, c_prediction = mlp_predict(userId)
    
    movies = pd.DataFrame({
        'movieId': bin_tags['movieId'],
        'r_prediction': r_prediciton,
        'c_prediction': c_prediction
    })
    
    # Remover os filmes que o usuário já avaliou
    user_rated_movies = train[train['userId'] == userId]['movieId']
    recommended_movies = movies[~movies['movieId'].isin(user_rated_movies)]
    
    # Ordenar os filmes pelas previsões e exibir os top k
    recommended_movies = recommended_movies.sort_values(by='r_prediction', ascending=False)
    
    # Taking top k movies that have the highest predicted ratings
    r_recomendations = recommended_movies[['r_prediction', 'c_prediction']].head(k)
    
    # Taking random k movies that are classified as relevant
    c_recomendations = recommended_movies[recommended_movies['c_prediction'] == True][['r_prediction', 'c_prediction']].sample(k)
    
    return r_recomendations, c_recomendations

In [46]:
recommend_movies(5928, k=10)

(     r_prediction  c_prediction
 39       4.021745          True
 38       3.997577          True
 80       3.955783          True
 101      3.493760          True
 95       3.482461          True
 55       3.480155          True
 78       3.023369          True
 10       1.561529          True
 6        1.490442          True
 3        1.281475          True,
      r_prediction  c_prediction
 23       0.385671          True
 54       0.302354          True
 169      0.302208          True
 166      0.301923          True
 154      0.301726          True
 227      0.449003          True
 10       1.561529          True
 163      0.436635          True
 21       0.719649          True
 66       0.444474          True)

Para esse usuário, podemos observar que a predição com o Regressor é mais consistente e resulta em avaliações preditas maiores. Isso é esperado, pois, nosso classificador binário apenas tenta prever se o item é relevante ou não. Observe que, no caso do usuário em análise, o regressor e o classificador discordam com frequência acerca de se um item é relevante ou não. Dado a caracteristica contínua do regressor, julgo que ele é mais útil na tarefa de classificação, visto que para além de detectar a relevância de um item, ele é capaz de comparar dois itens e determinar qual é mais relevante.